In [1]:
import chess
import pandas as pd
import numpy as np
import time
import positional_features
from IPython.display import clear_output
from stockfish import Stockfish

CSV_FILE_PATH = "../data/game_data.csv"
USER = "matei_popescu1510"
stockfish = Stockfish(path = "C:\Program Files\stockfish\stockfish-windows-x86-64-avx2.exe", depth=15)

In [2]:
game_data = pd.read_csv(CSV_FILE_PATH)
positions = {}

In [ ]:
# ## Testing
# board = chess.Board()
# moves = game_data['moves'][11]
# white_id = game_data['white_id'][11]
# black_id = game_data['black_id'][11]
# user_side = chess.WHITE if white_id == USER else chess.BLACK

# for move in moves.split():
#     if board.turn == user_side:
#         fen = board.fen()
#         if fen not in positions:
#             positions[fen] = set()
#         positions[fen].add(move)
#     board.push_uci(move)
    
# print(positions)

In [3]:
# TODO augment dataset
for index, row in game_data.iterrows():
    board = chess.Board()
    moves = row['moves'].split()
    white_id = row['white_id']
    black_id = row['black_id']
    user_side = chess.WHITE if white_id == USER else chess.BLACK
    
    for move in moves:
        if board.turn == user_side:
            fen = board.fen()
            if fen not in positions:
                positions[fen] = {}
            positions[fen][move] = positions[fen].get(move, 0) + 1
        board.push_uci(move)

positions_tuples = [(fen, move, counter) for fen, moves in positions.items() for move, counter in moves.items()] 
# if counter > 1 /// If this condition is added, all unique positions I've been in drop from over 100000 to around 2400. Goes to show how thin the tree of positions gets the further a game goes.
dataset = pd.DataFrame(positions_tuples, columns=['fen', 'move','counter'])

In [ ]:
POSITIONS_CSV_PATH = '../data/dataset.csv'
dataset.to_csv(POSITIONS_CSV_PATH)
dataset